In [8]:
load('anemoi.sage')
load('models.sage')
load('SystemAnalysis.sage')

# Anemoi over prime fields

In [9]:
p8 = 2^8 - 2^2 - 1
p17 = 2^16 + 1
p32 = 2^32 - (2^8 - 2^6 + 2^4 + 1) # = 2^32-209 # 2^32-2^4-1 # 2^31 - 1
p64 = 2^64 - (2^8 + 2^6 + 2^5 + 1) # = 2^64 - 353 # 2^64 - 2^32 + 1
#primes = [p1,p2,p3, PALLAS_BASEFIELD, VESTA_BASEFIELD, BLS12_377_SCALARFIELD, BLS12_381_SCALARFIELD, BN_254_SCALARFIELD]
primes = [p8,p17,p32,p64, PALLAS_BASEFIELD, VESTA_BASEFIELD, BLS12_377_SCALARFIELD, BLS12_381_SCALARFIELD, BN_254_SCALARFIELD]

alphas = [3,5,7,9,11]
result = [['p', 'hex(p)', 'b'] + alphas]

for p in primes:
    assert(is_prime(p))
    result.append([p, hex(p), ceil(log(p,2))] + [gcd(p-1,a) == 1 for a in alphas])
table(result, header_row=True, header_column=True)

p,hex(p),b,,,,,
,0xfb,,,,,,
,0x10001,,,,,,
,0xffffff2f,,,,,,
,0xfffffffffffffe9f,,,,,,
,0x40000000000000000000000000000000224698fc094cf91b992d30ed00000001,,,,,,
,0x40000000000000000000000000000000224698fc0994a8dd8c46eb2100000001,,,,,,
,0x12ab655e9a2ca55660b44d1e5c37b00159aa76fed00000010a11800000000001,,,,,,
,0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001,,,,,,
,0x30644e72e131a029b85045b68181585d2833e84879b9709143e1f593f0000001,,,,,,


In [103]:
Ap = AnemoiPermutation(q=p0, alpha=3, n_rounds=2, n_cols=1); print(Ap)  # Anemoi instance for some 8bit prime

Anemoi instance over F_251 (odd prime field), n_rounds=2, n_cols=1, s=128
alpha=3, beta=6, \delta=42
M_x=
[1]
C=[[14], [234]]
D=[[56], [110]]


In [104]:
# Check implementation of inverse linear layer
for x in Ap.F:
    for y in Ap.F:
        u, v = Ap.linear_layer([x],[y])
        x_, y_ = Ap.inv_linear_layer(u, v)
        assert([x] == x_[:] and [y] == y_[:])

In [105]:
# Bruteforce CICO solutions
for p in Ap.F:
    instate  = [Ap.F.zero(), p]
    outstate = Ap(instate)
    if outstate[0] == Ap.F.zero():
        print(f"A{instate} = {outstate}")

A[0, 170] = [0, 37]
A[0, 175] = [0, 96]


In [106]:
# Create equations for two models
fcico_Ap, _ =  model_F_CICO(Ap, n_rounds=2, debug=True)
pcico_Ap, _ =  model_P_CICO(Ap, n_rounds=2, debug=True)


Anemoi instance over F_251 (odd prime field), n_rounds=2, n_cols=1, s=128
alpha=3, beta=6, \delta=42
M_x=
[1]
C=[[14], [234]]
D=[[56], [110]]
----------------------------------------------------------------------------------------------------
Multivariate Polynomial Ring in X0000, Y0000, X0100, Y0100, X0200, Y0200 over Finite Field of size 251
[[X0000], [X0100], [X0200]]
[[Y0000], [Y0100], [Y0200]]
----------------------------------------------------------------------------------------------------
Number of equations:  4
Number of variables:  6
----------------------------------------------------------------------------------------------------
Degrees: [3, 3, 3, 3]
----------------------------------------------------------------------------------------------------
CICO variables:  {X0000: 0, X0200: 0}
Apply ordering 1
here
change ring for polys
change ring for variables
----------------------------------------------------------------------------------------------------
Number of equat

In [75]:
V_fcico_Ap = Ideal(fcico_Ap).variety(); V_fcico_Ap

[{Y0200: 96, Y0100: 203, Y0000: 175, X0100: 104},
 {Y0200: 37, Y0100: 25, Y0000: 170, X0100: 57}]

In [76]:
V_pcico_Ap = Ideal(pcico_Ap).variety(); V_pcico_Ap 

[{S0100: 42, S0200: 208, Y0000: 175}, {S0100: 215, S0200: 101, Y0000: 170}]

In [77]:
for v in V_fcico_Ap:
    result = Ap.eval_with_intermediate_values([Ap.F.zero()],[v['Y0000']])
    print(result)

[[[0], [175]], [[104], [203]], [[155], [192]], [[0], [96]]]
[[[0], [170]], [[57], [25]], [[214], [74]], [[0], [37]]]


# Anemoi over binary fields

In [6]:
alphas = [(i,2^i+1) for i in range(1,4)] # construction of alpha: alpha = 2^i + 1
powers = [7, 15, 17, 31, 63, 65, 127, 255, 257] # n for field 2^n

result = [['n'] + [a for (i,a) in alphas]]

for n in powers:
    result.append([n] + [(n%2 == 1) and (gcd(i,n) == 1) for (i,a) in alphas])
table(result, header_row=True, header_column=True)

n,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,


In [130]:
A2 = AnemoiPermutation(q=2^7, alpha=3, n_rounds=2, n_cols=1); print(A2)  # Anemoi instance for 7bit power of two

Anemoi instance over F_128 (characteristic 2), n_rounds=2, n_cols=1, s=128
alpha=3, beta=z7, \delta=z7^6 + 1
M_x=
[1]
C=[[2], [41]]
D=[[67], [63]]


In [131]:
# Check implementation of inverse linear layer
for x in A2.F:
    for y in A2.F:
        u, v = A2.linear_layer([x],[y])
        x_, y_ = A2.inv_linear_layer(u, v)
        assert([x] == x_[:] and [y] == y_[:])

In [132]:
# Bruteforce CICO solutions
for p in A2.F:
    instate  = [A2.F.zero(), p]
    outstate = A2(instate)
    if outstate[0] == A2.F.zero():
        print(f"A{instate} = {outstate}")

A[0, z7^6 + z7^5 + z7^4 + z7^2] = [0, z7^4]
A[0, z7^5 + 1] = [0, z7^4 + z7^2 + z7 + 1]


In [133]:
# Create equations for two models
fcico_A2, _ =  model_F_CICO(A2, n_rounds=2, debug=True)
pcico_A2, _ =  model_P_CICO(A2, n_rounds=2, debug=True)


Anemoi instance over F_128 (characteristic 2), n_rounds=2, n_cols=1, s=128
alpha=3, beta=z7, \delta=z7^6 + 1
M_x=
[1]
C=[[2], [41]]
D=[[67], [63]]
----------------------------------------------------------------------------------------------------
Multivariate Polynomial Ring in X0000, Y0000, X0100, Y0100, X0200, Y0200 over Finite Field in z7 of size 2^7
[[X0000], [X0100], [X0200]]
[[Y0000], [Y0100], [Y0200]]
----------------------------------------------------------------------------------------------------
Number of equations:  4
Number of variables:  6
----------------------------------------------------------------------------------------------------
Degrees: [3, 3, 3, 3]
----------------------------------------------------------------------------------------------------
CICO variables:  {X0000: 0, X0200: 0}
Apply ordering 1
here
change ring for polys
change ring for variables
----------------------------------------------------------------------------------------------------
Numb

In [134]:
V_fcico_A2 = Ideal(fcico_A2).variety(); V_fcico_A2

[{Y0200: z7^4 + z7^2 + z7 + 1, Y0100: z7^6 + z7^2 + z7, Y0000: z7^5 + 1, X0100: z7^3 + z7^2 + z7 + 1},
 {Y0200: z7^4, Y0100: z7^6 + z7^3 + z7^2 + z7 + 1, Y0000: z7^6 + z7^5 + z7^4 + z7^2, X0100: z7^5 + z7^3 + 1}]

In [135]:
V_pcico_A2 = Ideal(pcico_A2).variety(); V_pcico_A2 

[{S0100: z7^5 + z7^2 + z7, S0200: z7^6 + z7^4 + z7^3 + z7^2 + z7 + 1, Y0000: z7^5 + 1},
 {S0100: z7^6 + z7^5 + z7^4 + z7^3 + z7, S0200: z7^6 + z7^5 + z7^4, Y0000: z7^6 + z7^5 + z7^4 + z7^2}]